In [3]:
import numpy as np
import pandas as pd
import ast

df_road = pd.read_csv("../data/road.csv")
df_traj = pd.read_csv("../fmm/traj_for_fmm.csv")
df_fmm = pd.read_csv("../fmm/fmm_all_fields.csv", sep=';')

output = "./fmm_result.html"

x_bias = 0.0061
y_bias = 0.0013

In [2]:
before = \
'''
<!doctype html>
<html>

<head>
  <meta charset="utf-8">
  <meta http-equiv="X-UA-Compatible" content="IE=edge">
  <meta name="viewport" content="initial-scale=1.0, user-scalable=no, width=device-width">
  <title>HELLO，AMAP!</title>
  <link rel="stylesheet" href="https://a.amap.com/jsapi_demos/static/demo-center/css/demo-center.css" />
  <style>
    html,
    body,
    #container {
      height: 100%;
      width: 100%;
    }

    .amap-icon img,
    .amap-marker-content img {
      width: 25px;
      height: 34px;
    }

    .cus_info_window {
      background-color: #fff;
      padding: 10px;
    }
  </style>
</head>

<body>
  <div id="container"></div>
  <script type="text/javascript" src="https://webapi.amap.com/maps?v=2.0&key=692a38180fd6dce577d2e206e522d8e3"></script>
  <script type="text/javascript">
    // 创建地图实例
    var map = new AMap.Map("container", {
      zoom: 13,
      center: [116.39, 39.92],
      resizeEnable: true
    });

    // 创建点覆盖物
    var marker = new AMap.Marker({
      position: new AMap.LngLat(116.39, 39.92),
      icon: '//a.amap.com/jsapi_demos/static/demo-center/icons/poi-marker-default.png',
      offset: new AMap.Pixel(-13, -30)
    });
    map.add(marker);

    // 创建信息窗体
    var infoWindow = new AMap.InfoWindow({
      isCustom: true,  // 使用自定义窗体
      content: '<div class="cus_info_window">HELLO,AMAP!</div>', // 信息窗体的内容可以是任意 html 片段
      offset: new AMap.Pixel(16, -45)
    });
    var onMarkerClick = function (e) {
      infoWindow.open(map, e.target.getPosition()); // 打开信息窗体
      // e.target 就是被点击的 Marker
    }

    marker.on('click', onMarkerClick); // 绑定 click 事件
'''
after = \
'''
  </script>
</body>

</html>
'''

In [10]:
def parse_linestring(s:str):
    first_num = 11
    s = s[first_num:-1]
    points = s.split(',')
    ret = list(map(lambda x: ast.literal_eval("[{}]".format(x.replace(' ',',',1))), points))
    return ret

def parse_cpath(s:str):
    return list(map(int, s.split(',')))

In [26]:
# 普通道路是蓝色，被匹配到的道路是红色.
matched_road_flag = np.zeros((40000), dtype=np.int8)
visual_roads = []
visual_matched_roads = []

visual_unmatch_trajs = []

# 红色的被匹配到的几何. 整个画太大了，所以放弃. 变成把原本的路图红色.
for idx, matched_traj in df_fmm.iterrows():
    if pd.isnull(matched_traj['cpath']):
        continue
    cpath = parse_cpath(matched_traj["cpath"])
    for p in cpath:
        matched_road_flag[p] = 1

# 蓝色的road
for idx, road in df_road.iterrows():
    roadid = int(road[0])
    geom = ast.literal_eval(road[1])
    geom = list(map(lambda x: [x[0]+x_bias, x[1]+y_bias], geom))
    if matched_road_flag[roadid] == 1:
        visual_matched_roads.append(geom)
    else:
        visual_roads.append(geom)

# 绿色虚线的原始轨迹.
for idx, traj in df_traj.iterrows():
    geom = traj['geom']
    geom = parse_linestring(geom)
    geom = list(map(lambda x: [x[0]+x_bias, x[1]+y_bias], geom))
    visual_unmatch_trajs.append(geom)

with open(output, "w", encoding="utf-8-sig") as f:
    f.write(before)
    middle = \
'''
    var lineArr = {};
    var polyline = new AMap.Polyline({{
      path: lineArr,          // 设置线覆盖物路径
      strokeColor: "#3344FF", // 线颜色
      strokeWeight: 5,        // 线宽
      strokeStyle: "solid",   // 线样式
    }});
    map.add(polyline);
'''.format(visual_roads)
    f.write(middle)
    middle = \
'''
    var lineArr = {};
    var polyline = new AMap.Polyline({{
      path: lineArr,          // 设置线覆盖物路径
      strokeColor: "#FF0000", // 线颜色
      strokeWeight: 5,        // 线宽
      strokeStyle: "solid",   // 线样式
    }});
    map.add(polyline);
'''.format(visual_matched_roads)
    f.write(middle)
#     middle = \
# '''
#     var lineArr = {};
#     var polyline = new AMap.Polyline({{
#       path: lineArr,          // 设置线覆盖物路径
#       strokeColor: "#FF00FF", // 线颜色
#       strokeWeight: 1,        // 线宽
#       strokeStyle: "dash",   // 线样式
#     }});
#     map.add(polyline);
# '''.format(visual_unmatch_trajs)
#     f.write(middle)
    f.write(after)

C:\Users\admin\AppData\Local\Temp\ipykernel_4060\384334685.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  roadid = int(road[0])
C:\Users\admin\AppData\Local\Temp\ipykernel_4060\384334685.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  geom = ast.literal_eval(road[1])


In [35]:
# 画出一条具体的轨迹和匹配结果.
target_tid = 10000
visual_unmatched = []
visual_matched = []

for idx, traj in df_traj.iterrows():
    tid = int(traj[0])
    if tid == target_tid:
        geom = parse_linestring(traj['geom'])
        geom = list(map(lambda x: [x[0]+x_bias, x[1]+y_bias], geom))
        visual_unmatched.append(geom)
        break

for idx, fmmentry in df_fmm.iterrows():
    tid = int(fmmentry['id'])
    if tid == target_tid:
        geom = parse_linestring(fmmentry['mgeom'])
        geom = list(map(lambda x: [x[0]+x_bias, x[1]+y_bias], geom))
        visual_matched.append(geom)
        break

with open(output, "w", encoding='utf-8-sig') as f:
    f.write(before)
    middle = \
'''
    var lineArr = {};
    var polyline = new AMap.Polyline({{
      path: lineArr,          // 设置线覆盖物路径
      strokeColor: "#FF00FF", // 线颜色
      strokeWeight: 3,        // 线宽
      strokeStyle: "dash",   // 线样式
    }});
    map.add(polyline);
'''.format(visual_unmatched)
    f.write(middle)

    middle = \
'''
    var lineArr = {};
    var polyline = new AMap.Polyline({{
      path: lineArr,          // 设置线覆盖物路径
      strokeColor: "#FF0000", // 线颜色
      strokeWeight: 5,        // 线宽
      strokeStyle: "dash",    // 线样式
    }});
    map.add(polyline);
'''.format(visual_matched)
    f.write(middle)
    f.write(after)

C:\Users\admin\AppData\Local\Temp\ipykernel_4060\1321351845.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  tid = int(traj[0])
